<h1 style ="font-size:3rem;color:orange;"> P6: Métodos Auxiliares </h1>

In [67]:
import numpy as np
import math
import statistics as stat
import random

# variables que probablemente sean parámetros de simul_main
num_maquinas_sist = 10
num_maquinas_adic = 4
num_operarios = 4
T = 1000*60 

hist_fallos = np.genfromtxt('E6.fallos.txt', dtype='float')
media_fallos = np.mean(hist_fallos)
desv_fallos = np.std(hist_fallos)
print("Distribución Normal de media: " + str(media_fallos.round(3)) + " horas y desviación estándar: " + str(desv_fallos.round(3)) + " horas")

Distribución Normal de media: 2.798 horas y desviación estándar: 0.201 horas


In [68]:
    # Variables globales
    t = tsuc = Tp = 0
    # LISTA de 4 posiciones con valores tLL tS1 tS2 tS3 
    # registro de tiempos en los que ocurrirán los sucesos
    LISTA = []
    for i in range(num_maquinas_sist + num_operarios):
      LISTA.append(10e9)
    # variables contador entradas y salidas opcionales para este ejercicios
    # numero de maquinas en cada nodo/operario variable de estado
    #### buscar forma de que dependan de num_operarios
    cola_operarios = 0
    operarios_trabajando = 0
    ### quizá hacer aquí la cola
    # variables de estado específicas
    sist_func = True    
    end_simulation = False
    end_operarios = False
    # variables específicas para métricas de esta práctica
    # instante en el que el sistema deja de funcionar o empieza a funcionar de nuevo
    # instante en el que los operarios empiezan/dejan de trabajar simultáneamente
    cambio_func_sist = cambio_oper_simult = 0 
    # tiempos totales
    acumulo_sist_no_func = acumulo_oper_simult = 0

## Funciones estadísticas

In [69]:
# genera un valor random según una distribución normal
def genera_normal(media,desv):
    normal = math.trunc(np.random.normal(media,desv)*60)
    return normal

# prueba de que funciona
print('Media en minutos: ' + str(math.trunc(media_fallos*60)))
print('Desviación en minutos: ' + str(math.trunc(desv_fallos*60)))
print('Valor random distribución normal: ' + str(genera_normal(media_fallos,desv_fallos)))
    
# metodo para obtener lambda en instante t (h)
def lambda_t(t):
    # variable linealmente
    lambda_i = 0.55
    lambda_f = 1.65
    m = (lambda_f-lambda_i)/T
    lambda_t = m*t + lambda_i
    return lambda_t

# prueba de que funciona
print("Lambda al inicio de la simulación es " + str(lambda_t(0)) + 
      ", al final es " + str(lambda_t(T)) + ' y a la mitad es ' + str(lambda_t(T/2)))

# genera un valor random según una distribución exponencial
def genera_exponencial(param_lambda): 
    exponencial = math.trunc(np.random.exponential(param_lambda)*60)
    return exponencial

# prueba de que funciona
print('Valor random exponencial al principio de la simulación: ' + str(genera_exponencial(lambda_t(T))))
print('Valor random exponencial a la mitad de la simulación: ' + str(genera_exponencial(lambda_t(T/2))))
print('Valor random exponencial al final de la simulación: ' + str(genera_exponencial(lambda_t(T))))

Media en minutos: 167
Desviación en minutos: 12
Valor random distribución normal: 177
Lambda al inicio de la simulación es 0.55, al final es 1.65 y a la mitad es 1.1
Valor random exponencial al principio de la simulación: 112
Valor random exponencial a la mitad de la simulación: 63
Valor random exponencial al final de la simulación: 71


In [70]:
def rutina_fallo_maquina(tsuc):
    global num_maquinas_adic, num_maquinas_sist, t, LISTA, cola_operarios, operarios_trabajando
    global cambio_func_sist, cambio_oper_simult, acumulo_oper_simult, acumulo_sist_no_func
    global sist_func, end_simulation, end_operarios
    
    print('----Falla una máquina en el minuto '+ str(tsuc) + '.----')
    num_maquinas_sist -= 1
    if (num_maquinas_adic > 0): 
        print('----Había ' + str(num_maquinas_adic) + ' máquinas de reserva.----')
        num_maquinas_adic -= 1
        num_maquinas_sist += 1
        print('----Ahora quedan ' + str(num_maquinas_adic) + ' máquinas de reserva.----')
    elif (num_maquinas_adic == 0) and (num_maquinas_sist > 0):
      print('----No quedan máquinas de reserva.----')
      if (sist_func == True):
        sist_func = False
        cambio_func_sist = tsuc
        print('----El sistema acaba de dejar de funcionar.----')
      print('----Ahora sólo hay ' + str(num_maquinas_sist) + ' máquinas funcionales. El sistema no puede funcionar.----')
    else:
      print('----No quedan máquinas en el sistema, no puede fallar ninguna más.----')
      return 0
    
    # se la atiende directamente
    print('----Hay '+ str(num_operarios-operarios_trabajando)+' operarios libres.----')
    if (cola_operarios == 0) and (operarios_trabajando < num_operarios):
      operarios_trabajando += 1
      lista_aux_oper = LISTA[-3:]
      LISTA[10 + lista_aux_oper.index(10e9)] = tsuc + genera_exponencial(lambda_t(tsuc))
      print('----El operario tardará '+ str(LISTA[10 + lista_aux_oper.index(10e9)]-tsuc) + ' minutos en arreglar la máquina.----')
      print('----Estará lista en el minuto ' + str(LISTA[10 + lista_aux_oper.index(10e9)]) + '.----')
      if (operarios_trabajando == 3):
        print('----Ahora todos los operarios están ocupados.----')
        cambio_oper_simult = tsuc
    # a la cola  
    elif (cola_operarios >= 0) and (operarios_trabajando == 3):
      print('----Todos los operarios están ocupados. La máquina se pone a la cola.----')
      cola_operarios += 1
      print('----Ahora hay ' + str(cola_operarios) + ' máquinas en la cola.----')

    t = tsuc
    Y = genera_normal(media_fallos,desv_fallos)
    t_prox_fallo_maq = t + Y
    #print('----El tiempo de : ' + str(t_prox_fallo_maq)+ '.----')
    if (t_prox_fallo_maq < T) and (num_maquinas_adic > 0):
      # en que hueco de la lista lo pongo?
      # mirar en la lsita de los 10 primeros, cual es 10e9
      lista_aux_maq = LISTA[:10]
      LISTA[lista_aux_maq.index(10e9)] = t_prox_fallo_maq
    elif(t_prox_fallo_maq >= T): 
      end_simulation = True
      print('----La simulación está llegando a su fin, no van a romperse más máquinas.----') 

In [71]:
def rutina_arregla_operario(tsuc):

    global num_maquinas_adic, num_maquinas_sist, t, LISTA, cola_operarios, operarios_trabajando
    global cambio_func_sist, cambio_oper_simult, acumulo_oper_simult, acumulo_sist_no_func
    global sist_func, end_simulation, end_operarios
    
    t = tsuc
    operarios_trabajando -= 1
    lista_aux_oper = LISTA[-3:]
    #LISTA[10 + lista_aux_oper.index(tsuc)] = 10e9
    print('----Se ha terminado de arreglar una máquina en el minuto '+ str(t) +'.----')

    # para ir midiendo el tiempo de func del sistema
    if (sist_func == False):
        num_maquinas_sist += 1
        print('----La maquina se integra en el sistema, ahora hay '+ str(num_maquinas_sist) + ' máquinas funcionando.----')
        if (num_maquinas_sist == 10):
            sist_func = True
            print('----El sistema vuelve a funcionar!.----')
            acumulo_sist_no_func += tsuc - cambio_func_sist
            cambio_func_sist = 0
    else:
        num_maquinas_adic += 1
        print('----La maquina se guarda de reserva, ahora hay '+ str(num_maquinas_adic) + ' máquinas adicionales.----')

    if (cola_operarios == 0): # última máquina en la cola
        print('----No hay máquinas en la cola, el operario queda libre.----')
        if (cambio_oper_simult > 0):
          print('----Se acaba el trabajo simultáneo!.----')
          acumulo_oper_simult += tsuc - cambio_oper_simult
          cambio_oper_simult = 0
        if(end_simulation == True):
            end_operarios = True
            print('----Esa era la última máquina en la cola. Los operarios deja de trabajar en el minuto '+ str(t) +'.----')

    elif(cola_operarios > 0): # quedan clientes a la cola genero un nuevo tiempo de arreglo para el op1
        print('----Quedan '+ str(cola_operarios) + ' máquinas en la cola.----')
        cola_operarios -= 1
        operarios_trabajando += 1
        lista_aux_oper = LISTA[-3:]
        LISTA[10 + lista_aux_oper.index(10e9)] = tsuc + genera_exponencial(lambda_t(tsuc))
        print('----El operario tardará '+ str(LISTA[10 + lista_aux_oper.index(10e9)]-tsuc) + ' minutos en arreglar la máquina.----')
        print('----Estará lista en el minuto ' + str(LISTA[10 + lista_aux_oper.index(10e9)]) + '.----') 

In [72]:
def simul_main():
    
    global num_maquinas_adic, num_maquinas_sist, t, LISTA, cola_operarios, operarios_trabajando
    global cambio_func_sist, cambio_oper_simult, acumulo_oper_simult, acumulo_sist_no_func
    global sist_func, end_simulation, end_operarios

    # Variables globales
    t = tsuc = Tp = 0
    # LISTA de 4 posiciones con valores tLL tS1 tS2 tS3 
    # registro de tiempos en los que ocurrirán los sucesos
    LISTA = []
    for i in range(num_maquinas_sist + num_operarios):
      LISTA.append(10e9)
    print(LISTA)
    # variables contador entradas y salidas opcionales para este ejercicios
    # numero de maquinas en cada nodo/operario variable de estado
    #### buscar forma de que dependan de num_operarios
    cola_operarios = 0
    operarios_trabajando = 0
    ### quizá hacer aquí la cola
    # variables de estado específicas
    sist_func = True    
    end_simulation = False
    end_operarios = False
    # variables específicas para métricas de esta práctica
    # instante en el que el sistema deja de funcionar o empieza a funcionar de nuevo
    # instante en el que los operarios empiezan/dejan de trabajar simultáneamente
    cambio_func_sist = cambio_oper_simult = 0 
    # tiempos totales
    acumulo_sist_no_func = acumulo_oper_simult = 0
    
    for i in range(num_maquinas_sist):
        LISTA[i] = genera_normal(media_fallos,desv_fallos)
        if LISTA[i] > T:
          Tp = propor_sist_func = acumulo_oper_simult = 0
          return -1
    print(LISTA)
    #ipdb.set_trace(context=6)
    print("----La simulación da comienzo.----")
    while (LISTA[0]>0 or LISTA[1]>0 or LISTA[2]>0 or LISTA[3]>0):
    
    # 2 nodos = 2 rutinas = 2 ifs

      if (end_simulation == False) and (LISTA.index(min(LISTA)) < 10):
              tsuc = LISTA[LISTA.index(min(LISTA))]
              LISTA[LISTA.index(min(LISTA))] = 10e9
              rutina_fallo_maquina(tsuc)
      elif (end_operarios == False) and (LISTA.index(min(LISTA)) >= 10):
              tsuc = LISTA[LISTA.index(min(LISTA))]
              LISTA[LISTA.index(min(LISTA))] = 10e9
              rutina_arregla_operario(tsuc)  
      else: 
          break
      # fin del bucle while y de la simulación
    print(t)
    Tp = max(0,t-T)
    print('----El horizonte temporal de la simulación eran '+ str(round(T/60,2)) + 'h ~ '+str(T)+' minutos.----')
    print('----La simulación ha terminado completamente en el minuto '+ str(Tp) + ' ~ '+str(round(Tp/60,2))+'h.----')

    propor_sist_func = (T-acumulo_sist_no_func)*100/T
    print('----El sistema no ha funcionado durante '+ str(acumulo_sist_no_func) + ' minutos ~ '+str(round(acumulo_sist_no_func/60,2))+'h.----')
    print('----La proporción de tiempo que el sistema ha estado en funcionamiento '+ str(round(propor_sist_func,2)) + '%.----')
    print('----Los operarios han trabajado simultáneamente durante '+ str(acumulo_oper_simult) + ' minutos ~ '+str(round(acumulo_oper_simult/60,2))+'h.----')

In [73]:
simul_main()

[10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0]
[170, 174, 162, 169, 155, 177, 170, 168, 167, 162, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0]
----La simulación da comienzo.----
----Falla una máquina en el minuto 155.----
----Había 4 máquinas de reserva.----
----Ahora quedan 3 máquinas de reserva.----
----Hay 4 operarios libres.----
----El operario tardará 11 minutos en arreglar la máquina.----
----Estará lista en el minuto 166.----
----Falla una máquina en el minuto 162.----
----Había 3 máquinas de reserva.----
----Ahora quedan 2 máquinas de reserva.----
----Hay 3 operarios libres.----
----El operario tardará 49 minutos en arreglar la máquina.----
----Estará lista en el minuto 211.----
----Falla una máquina en el minuto 162.----
----Había 2 máquinas de reserva.----
----Ahora quedan 1 máquinas de reserva.----
